In [1]:
import numpy as np
import pandas as pd
import evaluate
import torch
import json
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
from torch.nn import CrossEntropyLoss

from preprocessing import climateBUG_reduce_rows



In [2]:
# This should always output true now, but worth checking
print(f"is GPU available: {torch.cuda.is_available()}")

is GPU available: True


In [3]:
# The climate-fever dataset is in a format which the transformers Trainer does not understand
# It must be preprocessed using the functions in preprocessing.py

train_df = pd.read_json("/home/lukeg/Documents/VS_code/fine_tuning/lxg406/climate_relatedness_classification/data/climateBUG/climateBUG-training-dataset.json")
preprocessed_training_df = climateBUG_reduce_rows(train_df, rows=10000)

test_df = pd.read_json("/home/lukeg/Documents/VS_code/fine_tuning/lxg406/climate_relatedness_classification/data/climateBUG/climateBUG-testing-dataset.json")
preprocessed_testing_df = climateBUG_reduce_rows(test_df, rows=1000)


                                              statement  label
0     2015REGISTRATION DOCUMENT ANNUAL FINANCIAL REP...      0
1     Retail     banking     achieved     a     stro...      0
2     Crédit   Agricole   Assurances   captured  fur...      0
3     Our  business  is  also  exposed  to  the  ups...      0
4     Crédit  Agricole  assumed  its   responsibilit...      0
...                                                 ...    ...
9995  Other general administrative expenses (Note 33...      0
9996  Other financial instruments not measured at fa...      0
9997             Revaluation gains/(losses) 191 (1,054)      0
9998  Final balance at 31.12.2014 5,714,956 12,032,8...      0
9999                Tangible assets (218,340) (249,701)      0

[10000 rows x 2 columns]
                                              statement  label
0     2015REGISTRATION DOCUMENT ANNUAL FINANCIAL REP...      0
1     Retail     banking     achieved     a     stro...      0
2     Crédit   Agricole   Ass

In [4]:
preprocessed_training_df.loc[0]

statement    2015REGISTRATION DOCUMENT ANNUAL FINANCIAL REP...
label                                                        0
Name: 0, dtype: object

In [5]:
preprocessed_testing_df.loc[0]

statement    4CRÉDIT AGRICOLE S.A. – 2020-2021 INTEGRATED R...
label                                                        0
Name: 0, dtype: object

In [6]:
training_dataset = Dataset.from_pandas(preprocessed_training_df)
testing_dataset = Dataset.from_pandas(preprocessed_testing_df)
training_dataset[0]

{'statement': '2015REGISTRATION DOCUMENT ANNUAL FINANCIAL REPORT2 CRÉDIT AGRICOLE S.A.      2015 REGISTRATION DOCUMENT MESSAGE FROM THE CHAIRMAN AND THE CHIEF EXECUTIVE OFFICER MESSAGE FROM THE CHAIRMANAND THE CHIEF EXECUTIVE OFFICERPHILIPPE BRASSAC, CHIEF EXECUTIVE OFFICER AND DOMINIQUE LEFEBVRE, CHAIRMAN OF CRÉDIT AGRICOLE S.A.A robust Group delivering sustainable performanceLeading  a  bank  with  a  EUR  698  billion   loan   book(1)',
 'label': 0}

In [7]:
seed=13
training_dataset = training_dataset.shuffle(seed=seed)
testing_dataset = testing_dataset.shuffle(seed=seed)

In [8]:
tokenizer = AutoTokenizer.from_pretrained("lumilogic/climateBUG-LM")

/home/lukeg/ClimatEnv/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
def custom_tokenize(examples):
    tokenized_output = tokenizer(
        text=examples["statement"],
        max_length=512, 
        padding="max_length", 
        truncation=True)

    return tokenized_output

tokenized_training_dataset = training_dataset.map(custom_tokenize, batched=True)
tokenized_testing_dataset = testing_dataset.map(custom_tokenize, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
tokenized_training_dataset

Dataset({
    features: ['statement', 'label', 'input_ids', 'attention_mask'],
    num_rows: 10000
})

In [11]:

print(tokenized_training_dataset[0]["statement"])
print(tokenized_training_dataset[0]["label"])
print(tokenized_training_dataset[0]["input_ids"])
print(tokenized_training_dataset[0]["attention_mask"])

print(tokenized_testing_dataset[9]["statement"])
print(tokenized_testing_dataset[9]["label"])
print(tokenized_testing_dataset[9]["input_ids"])
print(tokenized_testing_dataset[9]["attention_mask"])

ING is one of the most sustainable listed banks in the world.
1
[0, 1862, 16, 65, 9, 5, 144, 1437, 50296, 3147, 1520, 11, 5, 232, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(
    "lumilogic/climateBUG-LM",
    num_labels=2,
    # ignore_mismatched_sizes=True
).to("cuda")

In [13]:
print(next(model.parameters()).device)

cuda:0


In [14]:
model.gradient_checkpointing_enable()

In [15]:
def calculate_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")

    return {
        "accuracy": accuracy,
        "f1_score": f1,
    }

In [16]:
# This is where we set the hyperparameters
training_args = TrainingArguments(
    output_dir="./results/climateBUG-LM",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=200,
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="no",
    save_steps=500,
    fp16=True,                          # Use 16-bit floating point instead of 32 - makes computation faster
    warmup_ratio=0.05,                    # Allows the model to adapt a little
    # gradient_accumulation_steps=2       # Might help with OOM errors, if we have them
    learning_rate=3e-5,
    push_to_hub=False,
)

In [17]:
trainer  = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_training_dataset,
    tokenizer=tokenizer,
    eval_dataset=tokenized_testing_dataset,
    compute_metrics=calculate_metrics,
)

In [18]:
trainer.train()

  0%|          | 0/3750 [00:00<?, ?it/s]

{'loss': 0.1725, 'grad_norm': 1.8158129453659058, 'learning_rate': 7.5e-06, 'epoch': 0.04}
{'loss': 0.2113, 'grad_norm': 1.950812816619873, 'learning_rate': 1.5478723404255318e-05, 'epoch': 0.08}
{'loss': 0.205, 'grad_norm': 46.7353401184082, 'learning_rate': 2.345744680851064e-05, 'epoch': 0.12}
{'loss': 0.3244, 'grad_norm': 1.9899318218231201, 'learning_rate': 2.9924199887703537e-05, 'epoch': 0.16}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.4193623661994934, 'eval_accuracy': 0.876, 'eval_f1_score': 0.8805390480420461, 'eval_runtime': 9.0792, 'eval_samples_per_second': 110.141, 'eval_steps_per_second': 13.768, 'epoch': 0.16}
{'loss': 0.3636, 'grad_norm': 31.72575569152832, 'learning_rate': 2.9511510387422798e-05, 'epoch': 0.2}
{'loss': 0.3224, 'grad_norm': 7.147098064422607, 'learning_rate': 2.909039865244245e-05, 'epoch': 0.24}
{'loss': 0.2903, 'grad_norm': 23.710050582885742, 'learning_rate': 2.86692869174621e-05, 'epoch': 0.28}
{'loss': 0.2524, 'grad_norm': 9.251081466674805, 'learning_rate': 2.8248175182481753e-05, 'epoch': 0.32}


  0%|          | 0/125 [00:00<?, ?it/s]

...

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.40598833560943604, 'eval_accuracy': 0.911, 'eval_f1_score': 0.9105279425073604, 'eval_runtime': 9.155, 'eval_samples_per_second': 109.23, 'eval_steps_per_second': 13.654, 'epoch': 2.72}
{'loss': 0.0918, 'grad_norm': 30.335771560668945, 'learning_rate': 2.5772038180797305e-06, 'epoch': 2.76}
{'loss': 0.1316, 'grad_norm': 0.06056082248687744, 'learning_rate': 2.1560920830993826e-06, 'epoch': 2.8}
{'loss': 0.1243, 'grad_norm': 0.48794472217559814, 'learning_rate': 1.7349803481190342e-06, 'epoch': 2.84}
{'loss': 0.1095, 'grad_norm': 0.49720534682273865, 'learning_rate': 1.3138686131386862e-06, 'epoch': 2.88}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.41429442167282104, 'eval_accuracy': 0.915, 'eval_f1_score': 0.9151443700029751, 'eval_runtime': 9.1427, 'eval_samples_per_second': 109.377, 'eval_steps_per_second': 13.672, 'epoch': 2.88}
{'loss': 0.094, 'grad_norm': 0.03769858926534653, 'learning_rate': 8.92756878158338e-07, 'epoch': 2.92}
{'loss': 0.1309, 'grad_norm': 5.0508880615234375, 'learning_rate': 4.7164514317798987e-07, 'epoch': 2.96}
{'loss': 0.2327, 'grad_norm': 0.04252670332789421, 'learning_rate': 5.0533408197641775e-08, 'epoch': 3.0}
{'train_runtime': 1375.8684, 'train_samples_per_second': 21.804, 'train_steps_per_second': 2.726, 'train_loss': 0.21135257708231608, 'epoch': 3.0}


TrainOutput(global_step=3750, training_loss=0.21135257708231608, metrics={'train_runtime': 1375.8684, 'train_samples_per_second': 21.804, 'train_steps_per_second': 2.726, 'total_flos': 3974021959680000.0, 'train_loss': 0.21135257708231608, 'epoch': 3.0})

In [19]:
# trainer.save_model("./results/climateBERT-base/climateBUG/first_run")
# Please remember to delete model.safetensors BEFORE adding to git. Causes issues...
# Also it is probably not worth running this block until the model is worth keeping

In [20]:
# Metrics are not included in the save model so we need to save them separately
metrics = trainer.evaluate()
with open(f"./results/climateBUG-LM/eval_metrics_seed{seed}.json", "w") as output_file:
    json.dump(metrics, output_file)

  0%|          | 0/125 [00:00<?, ?it/s]